In [1]:
!pip install nltk


In [2]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
#from sklearn.linear_model import LogisticRegression
#from sklearn.metrics import accuracy_score

In [3]:
import nltk

In [4]:
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [5]:
train_set=pd.read_csv('train.csv')

In [6]:
train_set.shape

(20800, 5)

In [7]:
train_set.head(5)

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [8]:
#fake-1
#real-0

In [9]:
#counting the number of missing values in each coloumn 
train_set.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [10]:
#as data set is huge and we dont have a huge number of missing data we can fill some values in those missinf values

In [11]:
#replacing the null values with empty string 
train_set=train_set.fillna('')

In [12]:
#merging the author name and news title
train_set['content']=train_set['author']+' '+train_set['title']
print(train_set['content'])

0        Darrell Lucus House Dem Aide: We Didn’t Even S...
1        Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo...
2        Consortiumnews.com Why the Truth Might Get You...
3        Jessica Purkiss 15 Civilians Killed In Single ...
4        Howard Portnoy Iranian woman jailed for fictio...
                               ...                        
20795    Jerome Hudson Rapper T.I.: Trump a ’Poster Chi...
20796    Benjamin Hoffman N.F.L. Playoffs: Schedule, Ma...
20797    Michael J. de la Merced and Rachel Abrams Macy...
20798    Alex Ansary NATO, Russia To Hold Parallel Exer...
20799              David Swanson What Keeps the F-35 Alive
Name: content, Length: 20800, dtype: object


In [13]:
X=train_set.drop(columns='label',axis=1)
print(X)

          id                                              title  \
0          0  House Dem Aide: We Didn’t Even See Comey’s Let...   
1          1  FLYNN: Hillary Clinton, Big Woman on Campus - ...   
2          2                  Why the Truth Might Get You Fired   
3          3  15 Civilians Killed In Single US Airstrike Hav...   
4          4  Iranian woman jailed for fictional unpublished...   
...      ...                                                ...   
20795  20795  Rapper T.I.: Trump a ’Poster Child For White S...   
20796  20796  N.F.L. Playoffs: Schedule, Matchups and Odds -...   
20797  20797  Macy’s Is Said to Receive Takeover Approach by...   
20798  20798  NATO, Russia To Hold Parallel Exercises In Bal...   
20799  20799                          What Keeps the F-35 Alive   

                                          author  \
0                                  Darrell Lucus   
1                                Daniel J. Flynn   
2                             Consortiu

In [14]:
Y=train_set['label']
print(Y)

0        1
1        0
2        1
3        1
4        1
        ..
20795    0
20796    0
20797    0
20798    1
20799    1
Name: label, Length: 20800, dtype: int64


# Stemming

In [15]:
from nltk.stem.porter import PorterStemmer
port_stem=PorterStemmer()

In [16]:
def stemming(content):
    stemmed_content = re.sub('[^a-zA-Z]',' ',content)
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split()
    stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    stemmed_content = ' '.join(stemmed_content)
    return stemmed_content

In [17]:
train_set['content']=train_set['content'].apply(stemming)

In [18]:
print(train_set['content'])

0        darrel lucu hous dem aid even see comey letter...
1        daniel j flynn flynn hillari clinton big woman...
2                   consortiumnew com truth might get fire
3        jessica purkiss civilian kill singl us airstri...
4        howard portnoy iranian woman jail fiction unpu...
                               ...                        
20795    jerom hudson rapper trump poster child white s...
20796    benjamin hoffman n f l playoff schedul matchup...
20797    michael j de la merc rachel abram maci said re...
20798    alex ansari nato russia hold parallel exercis ...
20799                            david swanson keep f aliv
Name: content, Length: 20800, dtype: object


In [19]:
X=train_set['content'].values
Y=train_set['label'].values

In [20]:
print(X)

['darrel lucu hous dem aid even see comey letter jason chaffetz tweet'
 'daniel j flynn flynn hillari clinton big woman campu breitbart'
 'consortiumnew com truth might get fire' ...
 'michael j de la merc rachel abram maci said receiv takeov approach hudson bay new york time'
 'alex ansari nato russia hold parallel exercis balkan'
 'david swanson keep f aliv']


In [21]:
print(Y)

[1 0 1 ... 0 1 1]


# TF-IDF Vectorizer

In [22]:
#converting textual data to numercial data
vectorizer = TfidfVectorizer()
vectorizer.fit(X)

X = vectorizer.transform(X)

In [23]:
X=X.toarray()

In [24]:
#splitting dataset to training and test data
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state=0)

In [25]:
#reshapping the data
X_train=X_train.T
Y_train=Y_train.reshape(1,X_train.shape[1])

X_test=X_test.T
Y_test=Y_test.reshape(1,X_test.shape[1])


In [26]:
#sigmoid function 
def sigmoid(x):
    return 1/(1+np.exp(-x))

In [27]:
#matrix multiplication
def dot (A, B):
    
    R = np.empty([A.shape[0], B.shape[1]])
    for i in range(A.shape[0]):
        for j in range(B.shape[1]):
            R[i,j] = np.dot(A[i,:], B[:,j])
    return R




In [28]:
#logistic regression
def model(X,Y,learning_rate,iterations):
    m=X_train.shape[1]
    n=X_train.shape[0]
    
    W=np.zeros((n,1))
    B=0
    cost_list=[]
    
    for i in range(iterations):
        
        Z=dot(W.T,X) + B
        A=sigmoid(Z)
        
        cost=-(1/m)*np.sum(Y*np.log(A)  + (1-Y)*np.log(1-A))
        
        dW=(1/m)*np.dot(A-Y,X.T)
        dB=(1/m)*np.sum(A-Y)
        
        W=W-learning_rate*dW.T
        B=B-learning_rate*dB
        
        cost_list.append(cost)
        
        if(i%(iterations/10)==0):
            print("Cost after ",i,"iteration is : ",cost)
        
    return W,B,cost_list    

In [29]:
iterations=10000
learning_rate=1.5
W,B,cost_list=model(X_train,Y_train,learning_rate=learning_rate,iterations=iterations)

Cost after  0 iteration is :  0.6931471805599453
Cost after  1000 iteration is :  0.2716239643039524
Cost after  2000 iteration is :  0.1933688953736983
Cost after  3000 iteration is :  0.15617673525691733
Cost after  4000 iteration is :  0.13328207726645278
Cost after  5000 iteration is :  0.11735108479239767
Cost after  6000 iteration is :  0.10544144324229893
Cost after  7000 iteration is :  0.096107071816634
Cost after  8000 iteration is :  0.08854087948345758
Cost after  9000 iteration is :  0.08225125779751179


In [30]:
#accuracy
def acc(X,Y,W,B):
    Z=dot(W.T,X) + B
    A=sigmoid(Z)
    
    A=A>0.5
    
    A=np.array(A,dtype='int64')
    
    acc= (1-np.sum(np.abs(A-Y))/Y.shape[1])*100
    
    print("Accuracy of the model is : ",acc,"%")
    
    return A

In [31]:
A=acc(X_test,Y_test,W,B)

Accuracy of the model is :  97.78846153846153 %


In [32]:
from sklearn.metrics import confusion_matrix

In [33]:
confusion_matrix(Y_test[0],A[0])

array([[2479,   85],
       [  30, 2606]], dtype=int64)

# Output prediction

In [42]:
if(Y_test[0][15]==A[0][15]):
    print('Label matched')
else:
    print('Label did not match')

Label matched
